### 1. Load Libs

In [1]:
import math
import numpy as np
import tensorflow as tf
import seaborn as sb
import matplotlib.pyplot as plt
sb.set()

### 2. Generate Dataset

In [2]:
## quadratic function: y = 10 + x**2
def get_y(x):
    return np.power(x, 2) + 10 
## sample x
def sample_data(n=10000, scale=100):
    data = []
    x = scale*(np.random.random_sample((n,))-0.5)
    for i in range(n):
        yi = get_y(x[i])
        data.append([x[i], yi])
    return np.array(data)

### 3. Config networks

In [3]:
## generator input
def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])

## generator network
def generator(Z, hsize=[16, 16], reuse=False):
    with tf.variable_scope("GAN/Generator",reuse=reuse):
        for hiddle_size in hsize:
            Z = tf.layers.dense(Z, hiddle_size, activation = tf.nn.leaky_relu)
        out = tf.layers.dense(Z, 2) ## logits layer, before softmax
    return out

## discriminator network
def discriminator(X, hsize=[16, 16], reuse=False):
    with tf.variable_scope("GAN/Discriminator",reuse=reuse):
        for hiddle_size in hsize:
            X = tf.layers.dense(X, hiddle_size, activation = tf.nn.leaky_relu)
        X = tf.layers.dense(X,2)
        out = tf.layers.dense(X,1) ## logits layer, before softmax
    return out, X

### 4.  Optimization Pipeline 

In [4]:
## Input placeholder
X = tf.placeholder(tf.float32, [None,2])
Z = tf.placeholder(tf.float32, [None,2])

## Forward Pass
G_sample = generator(Z, hsize=[16,16])
r_logits, r_rep = discriminator(X, hsize=[16,16])
f_logits, g_rep = discriminator(G_sample, hsize=[16,16],reuse=True)

## Compute Loss
dis_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=r_logits, labels=tf.ones_like(r_logits)) + \
                          tf.nn.sigmoid_cross_entropy_with_logits(logits=f_logits, labels=tf.zeros_like(f_logits)))

gen_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=f_logits, labels=tf.ones_like(f_logits)))

## Trainables
gen_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="GAN/Generator")
dis_vars = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="GAN/Discriminator")

## Optimizer
lr = 1e-3
gen_step = tf.train.RMSPropOptimizer(learning_rate=lr).minimize(gen_loss, var_list = gen_vars) # G Train step
dis_step = tf.train.RMSPropOptimizer(learning_rate=lr).minimize(dis_loss, var_list = dis_vars) # D Train step

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.


In [5]:
# sess = tf.Session(config=config)
sess = tf.Session()
tf.global_variables_initializer().run(session=sess)

batch_size = 256
nd_steps = 10
ng_steps = 10

x_plot = sample_data(n=batch_size)

f = open('loss_logs.csv','w')
f.write('Iteration,Discriminator Loss,Generator Loss\n')

for i in range(10001):
    X_batch = sample_data(n = batch_size)
    Z_batch = sample_Z(batch_size, 2)

    ## run discriminator
    for _ in range(nd_steps):
        _, dloss = sess.run([dis_step, dis_loss], feed_dict={X: X_batch, Z: Z_batch})
        
    ## get representations
    rrep_dstep, grep_dstep = sess.run([r_rep, g_rep], feed_dict={X: X_batch, Z: Z_batch})

    ## run generator
    for _ in range(ng_steps):
        _, gloss = sess.run([gen_step, gen_loss], feed_dict={Z: Z_batch})

    ## get representations
    rrep_gstep, grep_gstep = sess.run([r_rep, g_rep], feed_dict={X: X_batch, Z: Z_batch})

    print ("Iterations: %d\t Discriminator loss: %.4f\t Generator loss: %.4f"%(i,dloss,gloss), end = '\r')
    
    if i%10 == 0:
        f.write("%d,%f,%f\n"%(i,dloss,gloss))

    if i%1000 == 0:
        
        ## plot Samples
        plt.figure()
        g_plot = sess.run(G_sample, feed_dict={Z: Z_batch})
        xax = plt.scatter(x_plot[:,0], x_plot[:,1])
        gax = plt.scatter(g_plot[:,0], g_plot[:,1])

        plt.legend((xax, gax), ("Real Data","Generated Data"))
        plt.title('Samples at Iteration %d'%i)
        plt.tight_layout()
        plt.savefig('../plots/iterations/iteration_%d.png'%i)
        plt.close()

        ## plot Features
        plt.figure()
        rrd = plt.scatter(rrep_dstep[:,0], rrep_dstep[:,1], alpha=0.5)
        rrg = plt.scatter(rrep_gstep[:,0], rrep_gstep[:,1], alpha=0.5)
        grd = plt.scatter(grep_dstep[:,0], grep_dstep[:,1], alpha=0.5)
        grg = plt.scatter(grep_gstep[:,0], grep_gstep[:,1], alpha=0.5)
        plt.legend((rrd, rrg, grd, grg), 
                   ("Real Data Before G step","Real Data After G step",
                    "Generated Data Before G step","Generated Data After G step"))
        plt.title('Transformed Features at Iteration %d'%i)
        plt.tight_layout()
        plt.savefig('../plots/features/feature_transform_%d.png'%i)
        plt.close()

        ## plot Generated data
        plt.figure()
        rrdc = plt.scatter(np.mean(rrep_dstep[:,0]), np.mean(rrep_dstep[:,1]),s=100, alpha=0.5)
        rrgc = plt.scatter(np.mean(rrep_gstep[:,0]), np.mean(rrep_gstep[:,1]),s=100, alpha=0.5)
        grdc = plt.scatter(np.mean(grep_dstep[:,0]), np.mean(grep_dstep[:,1]),s=100, alpha=0.5)
        grgc = plt.scatter(np.mean(grep_gstep[:,0]), np.mean(grep_gstep[:,1]),s=100, alpha=0.5)

        plt.legend((rrdc, rrgc, grdc, grgc), ("Real Data Before G step","Real Data After G step",
                               "Generated Data Before G step","Generated Data After G step"))

        plt.title('Centroid of Transformed Features at Iteration %d'%i)
        plt.tight_layout()
        plt.savefig('../plots/features/feature_transform_centroid_%d.png'%i)
        plt.close()

f.close()